In [1]:
import pyarrow.dataset as ds
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

In [2]:
from statsmodels.stats import stattools
from scipy import stats
import seaborn as sns
import psutil

In [3]:
from RiskLabAI.controller import Controller
# initialize controller
controller = Controller()

In [4]:
import optuna
from optuna.samplers import TPESampler
from optuna.exceptions import TrialPruned
from optuna.samplers import QMCSampler, CmaEsSampler
import torch
print(torch.backends.mps.is_available())

True


# Load Datasets

In [5]:
#Asset under study
ticker = 'BTCUSDT'

# define dataset
dataset = ds.dataset(
    "/Users/bobet/Documents/Code Repository/Trading-Systems/_datasets",
    format="parquet")

# push filter into Arrow scan (faster, uses partition pruning if possible)
table = dataset.to_table(filter=ds.field("symbol") == ticker)

# convert to pandasssss
df = table.to_pandas()
df.tail()

,symbol,ts_ms,iso_utc,ohlc_ts_open,ohlc_open,ohlc_high,ohlc_low,ohlc_close,ohlc_volume,ohlc_ts_close,...,tr_volume_base,tr_volume_quote,tr_vwap,tr_buy_sell_imbalance,spot_price,perp_mark_price,basis_abs,basis_pct,funding_rate,next_funding_time_ms
39162,BTCUSDT,1757821722142,2025-09-14T03:48:42.142655+00:00,1757821680000,115741.99,115742.00,115729.25,115729.26,3.68421,1757821739999,...,12.22828,1.415327e+06,115742.129388,-0.359820,115729.25,115674.594917,-54.655083,-0.000472,0.000082,1757836800000
39163,BTCUSDT,1757821782162,2025-09-14T03:49:42.162661+00:00,1757821740000,115729.25,115746.27,115729.25,115746.27,2.35966,1757821799999,...,7.30407,8.453633e+05,115738.666289,0.406802,115746.27,115704.300000,-41.970000,-0.000363,0.000082,1757836800000
39164,BTCUSDT,1757821842222,2025-09-14T03:50:42.222298+00:00,1757821800000,115746.27,115746.27,115746.26,115746.26,1.97252,1757821859999,...,5.76467,6.672202e+05,115743.000526,0.167002,115746.26,115705.200271,-41.059729,-0.000355,0.000082,1757836800000
39165,BTCUSDT,1757821902272,2025-09-14T03:51:42.272630+00:00,1757821860000,115742.13,115742.13,115740.04,115740.04,1.64508,1757821919999,...,7.64589,8.849185e+05,115737.800656,-0.275776,115740.04,115688.728758,-51.311242,-0.000443,0.000082,1757836800000
39166,BTCUSDT,1757821962322,2025-09-14T03:52:42.322623+00:00,1757821920000,115740.05,115740.05,115740.04,115740.04,1.43726,1757821979999,...,8.60731,9.962008e+05,115738.920133,0.084968,115740.04,115688.258712,-51.781287,-0.000447,0.000082,1757836800000


In [6]:
#features
df.columns

Index(['symbol', 'ts_ms', 'iso_utc', 'ohlc_ts_open', 'ohlc_open', 'ohlc_high',
       'ohlc_low', 'ohlc_close', 'ohlc_volume', 'ohlc_ts_close', 'ohlc_trades',
       'ohlc_taker_base', 'ohlc_taker_quote', 'l1_bid', 'l1_ask', 'l1_mid',
       'l1_spread', 'l1_bid_qty', 'l1_ask_qty', 'l1_imbalance', 'l2_bid_depth',
       'l2_ask_depth', 'l2_depth_asymmetry', 'l2_bid_vwap', 'l2_ask_vwap',
       'l2_bid_slope', 'l2_ask_slope', 'tr_volume_base', 'tr_volume_quote',
       'tr_vwap', 'tr_buy_sell_imbalance', 'spot_price', 'perp_mark_price',
       'basis_abs', 'basis_pct', 'funding_rate', 'next_funding_time_ms'],
      dtype='object')

In [7]:
sample_size = df.count()[0]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39167 entries, 0 to 39166
Data columns (total 37 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   symbol                 39167 non-null  object 
 1   ts_ms                  39167 non-null  int64  
 2   iso_utc                39167 non-null  object 
 3   ohlc_ts_open           39167 non-null  int64  
 4   ohlc_open              39167 non-null  float64
 5   ohlc_high              39167 non-null  float64
 6   ohlc_low               39167 non-null  float64
 7   ohlc_close             39167 non-null  float64
 8   ohlc_volume            39167 non-null  float64
 9   ohlc_ts_close          39167 non-null  int64  
 10  ohlc_trades            39167 non-null  int64  
 11  ohlc_taker_base        39167 non-null  float64
 12  ohlc_taker_quote       39167 non-null  float64
 13  l1_bid                 39167 non-null  float64
 14  l1_ask                 39167 non-null  float64
 15  l1

## Raw Features

### General
- **symbol**: Trading pair identifier (e.g., BTCUSDT).  
- **ts_ms**: Data timestamp in milliseconds (epoch time).  
- **iso_utc**: Data timestamp in human-readable UTC format.  

### OHLC Data (Candlestick)
- **ohlc_ts_open**: Opening timestamp for the candlestick period.  
- **ohlc_open**: Opening price of the candlestick.  
- **ohlc_high**: Highest price within the candlestick.  
- **ohlc_low**: Lowest price within the candlestick.  
- **ohlc_close**: Closing price of the candlestick.  
- **ohlc_volume**: Trading volume during the candlestick (base asset units).  
- **ohlc_ts_close**: Closing timestamp for the candlestick period.  
- **ohlc_trades**: Number of trades in the candlestick.  
- **ohlc_taker_base**: Base asset volume traded by takers (aggressors).  
- **ohlc_taker_quote**: Quote asset volume traded by takers.  

### Level 1 Order Book (Top of Book)
- **l1_bid**: Best bid price (highest buy order).  
- **l1_ask**: Best ask price (lowest sell order).  
- **l1_mid**: Midpoint price between bid and ask.  
- **l1_spread**: Difference between best ask and bid (ask - bid).  
- **l1_bid_qty**: Quantity available at best bid.  
- **l1_ask_qty**: Quantity available at best ask.  
- **l1_imbalance**: Order book imbalance at Level 1 = (bid_qty – ask_qty) / (bid_qty + ask_qty).  

### Level 2 Order Book (Depth of Market)
- **l2_bid_depth**: Total buy-side liquidity across multiple bid levels.  
- **l2_ask_depth**: Total sell-side liquidity across multiple ask levels.  
- **l2_depth_asymmetry**: Relative difference between bid and ask depth.  
- **l2_bid_vwap**: Volume-weighted average bid price across order book levels.  
- **l2_ask_vwap**: Volume-weighted average ask price across order book levels.  
- **l2_bid_slope**: Measure of how steeply bid prices rise with quantity (liquidity gradient).  
- **l2_ask_slope**: Measure of how steeply ask prices rise with quantity.  

### Trade Data
- **tr_volume_base**: Total traded volume in base asset.  
- **tr_volume_quote**: Total traded volume in quote asset.  
- **tr_vwap**: Trade volume-weighted average price.  
- **tr_buy_sell_imbalance**: Difference between buy-initiated and sell-initiated trade volumes.  

### Derived Prices
- **spot_price**: Current spot market price.  
- **perp_mark_price**: Mark price used in perpetual futures to avoid manipulation.  
- **basis_abs**: Absolute difference between perpetual mark price and spot price.  
- **basis_pct**: Percentage difference between perpetual mark price and spot price.  
- **funding_rate**: Periodic payment rate between long and short positions in perpetual contracts.  
- **next_funding_time_ms**: Timestamp (ms) of the next funding event.  

## Data hygiene & storage

✔ Why: storage efficiency + ordering. A 10GB dataset may shrink to ~3-4GB when optimized.

In [8]:
# Ensure correct dtypes (saves memory on 10GB dataset)
dtype_map = {
    "symbol": "category",
    "ohlc_open": "float32", "ohlc_high": "float32", "ohlc_low": "float32", "ohlc_close": "float32",
    "ohlc_volume": "float32", "ohlc_trades": "int32",
    "ohlc_taker_base": "float32", "ohlc_taker_quote": "float32",
    "l1_bid": "float32", "l1_ask": "float32", "l1_mid": "float32", "l1_spread": "float32",
    "l1_bid_qty": "float32", "l1_ask_qty": "float32", "l1_imbalance": "float32",
    "l2_bid_depth": "float32", "l2_ask_depth": "float32", "l2_depth_asymmetry": "float32",
    "l2_bid_vwap": "float32", "l2_ask_vwap": "float32",
    "l2_bid_slope": "float32", "l2_ask_slope": "float32",
    "tr_volume_base": "float32", "tr_volume_quote": "float32", "tr_vwap": "float32",
    "tr_buy_sell_imbalance": "float32",
    "spot_price": "float32", "perp_mark_price": "float32",
    "basis_abs": "float32", "basis_pct": "float32", "funding_rate": "float32"
}

df = df.astype(dtype_map)

# Make sure timestamp is datetime
df["iso_utc"] = pd.to_datetime(df["iso_utc"])
df = df.set_index("iso_utc").sort_index()


#Intergrity Check
# Drop duplicates, check ordering
df = df[~df.index.duplicated(keep="first")].sort_index()

# Sanity checks for OHLC
mask = (
    (df["ohlc_low"] <= df["ohlc_open"]) &
    (df["ohlc_low"] <= df["ohlc_close"]) &
    (df["ohlc_high"] >= df["ohlc_open"]) &
    (df["ohlc_high"] >= df["ohlc_close"])
)
df = df[mask]

# Check non-negative volumes
df = df[df["ohlc_volume"] >= 0]

## Convert DataFrame to RiskLA AI Input Format

In [9]:
df_riskAI = df.copy()
df_riskAI = df_riskAI.loc[:, ['symbol', 'ohlc_close', 'ohlc_volume']]
#rename column
df_riskAI.reset_index(inplace=True) 
df_riskAI.set_index('symbol', inplace=True)
df_riskAI.columns = ['date', 'price', 'volume']
df_riskAI.head()

,date,price,volume
symbol,,,
BTCUSDT,2025-08-17 15:45:47.575950+00:00,118251.351562,0.75580
BTCUSDT,2025-08-17 15:46:47.606265+00:00,118234.531250,5.14589
BTCUSDT,2025-08-17 15:47:47.643040+00:00,118234.531250,2.35213
BTCUSDT,2025-08-17 15:48:47.662644+00:00,118234.523438,6.03409
BTCUSDT,2025-08-17 15:49:47.702649+00:00,118234.523438,1.28289


## Load Existing Parameter Resampling Database

In [10]:
param_db_filename = 'params_resampling-scheme.json'
# Open and load JSON file

def load_param_resampling_db(file_name):
    """
    Load resampling parameter database from a JSON file.

    Parameters
    ----------
    file_name : str
        Path to the JSON file containing the parameter database.

    Returns
    -------
    list of dict
        A list of parameter dictionaries previously stored.
        If the file does not exist or is invalid, returns an empty list.

    Notes
    -----
    - The expected structure is a list of dictionaries where each dictionary
      contains keys such as 'Sampling_Scheme', 'n_trials', 'n_samples', etc.
    - Use together with `sampling_best_params` for managing optimal bar parameters.

    Examples
    --------
    >>> params_db = load_param_resampling_db("params_resampling-scheme.json")
    >>> len(params_db)
    5
    """
    with open(file_name, "r") as f:
        df_params = json.load(f)
    return df_params

db_resampling_params = load_param_resampling_db(param_db_filename)
db_resampling_params

[{'ticker': 'BTCUSDT',
  'time_frame': '1m',
  'Sampling_Scheme': 'expected_tick_imbalance_bars',
  'n_trials': 50,
  'n_samples': 39167,
  'batch_size': 30275161,
  'Shapiro-Wilk_statistic': 0,
  'Shapiro-Wilk_pvalue': 0,
  'window_size_for_expected_n_ticks_estimation': 5,
  'window_size_for_expected_imbalance_estimation': 2000,
  'initial_estimate_of_expected_n_ticks_in_bar': 50},
 {'ticker': 'BTCUSDT',
  'time_frame': '1m',
  'Sampling_Scheme': 'expected_volume_imbalance_bars',
  'n_trials': 50,
  'n_samples': 39167,
  'batch_size': 30275161,
  'Shapiro-Wilk_statistic': 0,
  'Shapiro-Wilk_pvalue': 0,
  'window_size_for_expected_n_ticks_estimation': 5,
  'window_size_for_expected_imbalance_estimation': 2000,
  'initial_estimate_of_expected_n_ticks_in_bar': 50},
 {'ticker': 'BTCUSDT',
  'time_frame': '1m',
  'Sampling_Scheme': 'expected_dollar_imbalance_bars',
  'n_trials': 50,
  'n_samples': 39167,
  'batch_size': 30275161,
  'Shapiro-Wilk_statistic': 0,
  'Shapiro-Wilk_pvalue': 0,
 

In [11]:
# df_param = pd.DataFrame(db_resampling_params)

# df_param.to_csv("params_resampling-scheme.csv")

In [12]:
# # Load CSV
# csv_file = "params_resampling-scheme.csv"
# df = pd.read_csv(csv_file)

# # Convert to JSON
# json_file = csv_file.replace(".csv", ".json")
# df.to_json(json_file, orient="records", indent=4)

# print(f"Saved JSON to {json_file}")

In [13]:
#load best parameters in the resampling database
def db_resampling_best_params(_sampling_method,db_params):
    """
    Retrieve the best parameter set for a given resampling method
    from the parameter database.

    Parameters
    ----------
    _sampling_method : str
        The sampling scheme to filter on, e.g.:
        - "expected_tick_imbalance_bars"
        - "expected_volume_run_bars"
    db_params : list of dict
        The resampling parameter database, typically loaded from JSON.

    Returns
    -------
    dict
        The best parameter dictionary for the given sampling scheme.

    Notes
    -----
    - Ranking is based on three criteria:
        1. `n_samples` (ascending) → prefer larger sample size.
        2. `Shapiro-Wilk_pvalue` (ascending).
        3. `Shapiro-Wilk_statistic` (ascending).
    - The last row after sorting is returned (`tail(1)`), so effectively it picks 
      the parameters with *highest* sample size, *highest* p-value, and *highest* statistic.

    Examples
    --------
    >>> best = db_resampling_best_params("expected_tick_imbalance_bars", params_db)
    >>> best["n_samples"]
    25000
    """
    db = pd.DataFrame(db_params)
    db = db.loc[db['Sampling_Scheme'] ==_sampling_method]

    # Sort by 'n_samples' first, then by 'Shapiro-Wilk_pvalue', then by 'Shapiro-Wilk_statistic'
    db = db.sort_values(
        by=['n_samples', 'Shapiro-Wilk_pvalue', 'Shapiro-Wilk_statistic'],
        ascending=[True, True, True])   # example: n_samples ↑, pvalue ↓, stat ↓

    return db.tail(1).to_dict(orient="records")[0]

In [14]:
db_best_params = db_resampling_best_params(_sampling_method='expected_dollar_imbalance_bars',db_params=db_resampling_params)
db_best_params

{'ticker': 'BTCUSDT',
 'time_frame': '1m',
 'Sampling_Scheme': 'expected_dollar_imbalance_bars',
 'n_trials': 50,
 'n_samples': 39167,
 'batch_size': 30275161,
 'Shapiro-Wilk_statistic': 0,
 'Shapiro-Wilk_pvalue': 0,
 'window_size_for_expected_n_ticks_estimation': 5,
 'window_size_for_expected_imbalance_estimation': 2000,
 'initial_estimate_of_expected_n_ticks_in_bar': 50}

# Utilily Functions

## Helper Functions

Summed (additive counts / volumes)

We sum over the ticks inside each bar:
	•	ohlc_trades
	•	ohlc_taker_base
	•	ohlc_taker_quote
	•	tr_volume_base
	•	tr_volume_quote

⸻

Last (state-like snapshots, take the latest tick in the bar)

We forward-fill within the bar only, then take the last available value:
	•	l1_bid, l1_ask, l1_mid, l1_spread
	•	l1_bid_qty, l1_ask_qty, l1_imbalance
	•	l2_bid_depth, l2_ask_depth, l2_depth_asymmetry
	•	l2_bid_vwap, l2_ask_vwap, l2_bid_slope, l2_ask_slope
	•	spot_price, perp_mark_price
	•	basis_abs, basis_pct
	•	funding_rate, next_funding_time_ms

(These are “levels” or “state variables” you’d want at the bar close.)

⸻

Mean (averaged inside the bar)
	•	tr_buy_sell_imbalance

⸻

Weighted mean (value × volume / total volume)
	•	tr_vwap (weighted by tr_volume_base)

(This is the standard definition of VWAP: average trade price weighted by base-asset volume.)

In [15]:
#Compute log returns from a price series.l
log_return = lambda s: np.log(s).diff().dropna()

# Hyper Parameter Optimization with Optuna

## Optimization_Helper Function

In [16]:
def _batch_size(df_raw):
    # Load a small sample of your data
    df_batch = df_raw.head(10000)  # or CSV, etc.

    # Estimate memory usage per row
    bytes_per_row = df_batch.memory_usage(deep=True).sum() / len(df_batch)

    # Get available RAM (bytes)
    avail_ram = psutil.virtual_memory().available

    # Budget: use only 30% of available RAM to be safe
    ram_budget = 0.3 * avail_ram

    # Rough batch size estimate
    batch_size = int(ram_budget / bytes_per_row)

    print(f"Bytes per row ≈ {bytes_per_row:.1f}")
    print(f"Available RAM ≈ {avail_ram/1e9:.1f} GB")
    print(f"Safe batch_size ≈ {batch_size:,} rows")

    return batch_size

In [17]:
def sampling_seach_space_parameter(input_df: pd.DataFrame) -> dict:
    """
    Classify irregular DatetimeIndex into timeframe bucket & trading style
    by using the median difference between consecutive timestamps.
    """
    if not isinstance(input_df.index, pd.DatetimeIndex):
        raise ValueError("DataFrame index must be a DatetimeIndex")
    
    # Compute median difference in seconds
    diffs = input_df.index.to_series().diff().dropna().dt.total_seconds()
    median_sec = diffs.median()
    
    # Convert to minutes
    minutes = median_sec / 60.0
    
    # Grid search for different trading styles

    scalper = {
        "wn":   [5, 10, 15],                 # window_size_for_expected_n_ticks_estimation
        "wi":   [1_000, 2_500, 5_000],     # window_size_for_expected_imbalance_estimation
        "seed": [50, 150],      # initial_estimate_of_expected_n_ticks_in_bar
    }

    day_trader = {
        "wn":   [10, 20, 30],
        "wi":   [5_000, 10_000, 20_000],
        "seed": [2_000, 10_000, 20_000],
    }

    swing_trader = {
        "wn":   [30, 50, 80],
        "wi":   [15_000, 30_000, 60_000],
        "seed": [10_000, 30_000, 60_000],
    }

    position_trader = {
        "wn":   [50, 80, 100],
        "wi":   [30_000, 80_000, 120_000],
        "seed": [20_000, 80_000, 120_000],
    }

    # Classification
    if minutes <= 5:
        bucket = f"{round(minutes)}m"
        style = "Scalper"
        search_space = scalper
    elif 10 <= minutes <= 60:
        bucket = f"{round(minutes)}m–1h"
        style = "Day trader"
        search_space = day_trader 
    elif 240 <= minutes <= 1440:
        bucket = f"{round(minutes/60)}h–1d"
        style = "Swing trader"
        search_space = swing_trader
    elif minutes >= 10080:  # 7 days
        bucket = "1w+"
        style = "Position trader"
        search_space =  position_trader
    else:
        bucket = f"{minutes:.1f}m"
        style = "Unclassified"
    
    return {
        "median_interval_min": minutes,
        "bucket": bucket,
        "style": style,
        "grid_search": search_space} 

In [18]:
# pip install pyswarms
import json
import numpy as np
from scipy import stats
import pyswarms as ps  # <-- PSO
# keep your existing imports for controller, db_resampling_best_params, etc.


# ---------- retention penalty ----------
def penalty(raw_df, sampling_output):
    """
    Penalize retention outside [0.15, 0.50].
    Returns (penalty, retention_ratio).
    """
    retention_ratio = len(sampling_output) / max(1, len(raw_df))
    if retention_ratio < 0.15:
        pen = (0.15 - retention_ratio) * 10
    elif retention_ratio > 0.50:
        pen = (retention_ratio - 0.50) * 10
    else:
        pen = 0.0
    return pen, retention_ratio


def sampling_best_params(
    sampling_method,
    df_sample,
    params_resampling_db,
    db_file_name,
    chunk_size,
    trials=50,              # PSO iters
    time_frame_label="1m",
    n_particles=24,         # PSO swarm size
    seed=123,
):
    """
    Tune hyperparameters for imbalance/run bar sampling using
    Jarque–Bera (JB) + retention penalty with Particle Swarm Optimization (PySwarms).
    """

    rng = np.random.default_rng(seed)
    n = len(df_sample)

    # --------- search ranges (same as your latest) ----------
    wn_range   = (1.5, 2_000)                            # window_size_for_expected_n_ticks_estimation
    wi_range   = (100, min(20_000, max(500, n // 10))) # window_size_for_expected_imbalance_estimation
    seed_range = (20, 1_000)                           # initial_estimate_of_expected_n_ticks_in_bar

    # We'll optimize in log10 space for all three, then exponentiate and round.
    lb = np.log10([wn_range[0],  wi_range[0],  seed_range[0]]).astype(float)
    ub = np.log10([wn_range[1],  wi_range[1],  seed_range[1]]).astype(float)

    # --------- preload existing best from DB (if any) ----------
    try:
        _db_best_params = db_resampling_best_params(
            _sampling_method=sampling_method,
            db_params=params_resampling_db,
        )
    except Exception:
        _db_best_params = None

    # --------- persist helper ----------
    def _save_to_db(_new_params: dict):
        params_resampling_db.append(_new_params)
        with open(db_file_name, "w") as f:
            json.dump(params_resampling_db, f, indent=4)

    # --------- selection helper ----------
    def _output(_old_param, _new_param, param_db, sample_size):
        if _old_param is None:
            _save_to_db(_new_param)
            return _new_param
        if _new_param in param_db:
            print("⚠️ Already exists.")
            return _old_param
        if _new_param.get("JB_pvalue") is None:
            print("ℹ️ Invalid trial, keeping existing best.")
            return _old_param
        if sample_size > _old_param.get("n_samples", 0):
            _save_to_db(_new_param)
            print("✅ Larger sample size, updated.")
            return _new_param
        if (sample_size == _old_param.get("n_samples", 0)
            and _new_param["JB_pvalue"] > _old_param.get("JB_pvalue", -1)):
            _save_to_db(_new_param)
            print("🔄 Same sample size, better JB p-value, updated.")
            return _new_param
        print("ℹ️ Did not satisfy update conditions — keeping existing best.")
        return _old_param

    # --------- PSO objective over the swarm ----------
    # x has shape (n_particles, 3) in log10-space
    BIG = 1e12  # big penalty for infeasible configs

    def _eval_single(log_params):
        # transform from log10 to int hyperparams (clip to bounds)
        wn  = int(np.clip(round(10 ** log_params[0]), wn_range[0], wn_range[1]))
        wi  = int(np.clip(round(10 ** log_params[1]), wi_range[0], wi_range[1]))
        seed_ticks = int(np.clip(round(10 ** log_params[2]), seed_range[0], seed_range[1]))

        # feasibility constraints (soft penalties):
        # give headroom to estimate imbalance vs tpb window, and avoid mega seeds
        if wi < 2 * wn:
            return BIG
        if seed_ticks > 5 * wn:
            return BIG

        try:
            bars = controller.handle_input_command(
                method_name=sampling_method,
                method_arguments={
                    "window_size_for_expected_n_ticks_estimation": wn,
                    "window_size_for_expected_imbalance_estimation": wi,
                    "initial_estimate_of_expected_n_ticks_in_bar": seed_ticks,
                },
                input_data=df_sample,
                batch_size=chunk_size,
            )
        except Exception:
            return BIG  # failed build → penalize

        # Minimum viable number of bars
        min_bars = max(200, n // 200)
        if len(bars) < min_bars:
            return BIG

        # Returns
        close_col = "Close" if "Close" in bars.columns else bars.columns[-1]
        rets = np.diff(np.log(bars[close_col].astype(float).to_numpy()))
        rets = rets[np.isfinite(rets)]
        if rets.size < 3 or np.var(rets) <= 1e-12:
            return BIG

        # JB + retention penalty
        jb_stat, _ = stats.jarque_bera(rets)
        ret_pen, _ = penalty(df_sample, bars)

        return float(jb_stat) + float(ret_pen)

    def pso_objective(X):
        # X: (n_particles, dim)
        return np.array([_eval_single(row) for row in X], dtype=float)

    # --------- set up initial swarm positions (optional warm start) ----------
    # default random in bounds:
    init_pos = rng.uniform(low=lb, high=ub, size=(n_particles, 3))

    # if DB best exists, seed first particle near it
    if _db_best_params is not None:
        try:
            seed_vec = np.log10([
                np.clip(int(_db_best_params["window_size_for_expected_n_ticks_estimation"]), *wn_range),
                np.clip(int(_db_best_params["window_size_for_expected_imbalance_estimation"]), *wi_range),
                np.clip(int(_db_best_params["initial_estimate_of_expected_n_ticks_in_bar"]), *seed_range),
            ])
            init_pos[0] = np.clip(seed_vec, lb, ub)
        except Exception:
            pass
    else:
        # seed center as a reasonable mid guess
        mid_vec = np.log10([
            (wn_range[0] + wn_range[1]) / 2,
            (wi_range[0] + wi_range[1]) / 2,
            (seed_range[0] + seed_range[1]) / 2,
        ])
        init_pos[0] = np.clip(mid_vec, lb, ub)

    # --------- run PSO ----------
    bounds = (lb, ub)
    options = dict(c1=0.5, c2=0.3, w=0.9)  # cognitive, social, inertia

    optimizer = ps.single.GlobalBestPSO(
        n_particles=n_particles,
        dimensions=3,
        options=options,
        bounds=bounds,
        init_pos=init_pos,
        ftol=1e-8,        # early stop tolerance on cost improvement
        ftol_iter=10,     # patience (iters)
    )

    best_cost, best_pos = optimizer.optimize(pso_objective, iters=trials, verbose=True)

    # --------- evaluate best solution to collect metrics ----------
    def _transform_and_eval(log_params):
        wn  = int(np.clip(round(10 ** log_params[0]), wn_range[0], wn_range[1]))
        wi  = int(np.clip(round(10 ** log_params[1]), wi_range[0], wi_range[1]))
        seed_ticks = int(np.clip(round(10 ** log_params[2]), seed_range[0], seed_range[1]))

        bars = controller.handle_input_command(
            method_name=sampling_method,
            method_arguments={
                "window_size_for_expected_n_ticks_estimation": wn,
                "window_size_for_expected_imbalance_estimation": wi,
                "initial_estimate_of_expected_n_ticks_in_bar": seed_ticks,
            },
            input_data=df_sample,
            batch_size=chunk_size,
        )

        # metrics
        close_col = "Close" if "Close" in bars.columns else bars.columns[-1]
        rets = np.diff(np.log(bars[close_col].astype(float).to_numpy()))
        rets = rets[np.isfinite(rets)]
        jb_stat, jb_pval = stats.jarque_bera(rets)
        ret_pen, ratio = penalty(df_sample, bars)
        return dict(wn=wn, wi=wi, seed_ticks=seed_ticks,
                    jb_stat=float(jb_stat), jb_pval=float(jb_pval),
                    retention_ratio=float(ratio),
                    loss=float(jb_stat) + float(ret_pen))

    best_metrics = _transform_and_eval(best_pos)
    sample_size = len(df_sample)

    new_params = {
        "ticker": df_sample.index.unique()[0] if len(df_sample.index) else "UNKNOWN",
        "time_frame": time_frame_label,
        "Sampling_Scheme": sampling_method,
        "n_trials": trials,
        "n_samples": sample_size,
        "batch_size": chunk_size,
        "JB_statistic": best_metrics["jb_stat"],
        "JB_pvalue": best_metrics["jb_pval"],
        "retention_ratio": best_metrics["retention_ratio"],
        "window_size_for_expected_n_ticks_estimation": best_metrics["wn"],
        "window_size_for_expected_imbalance_estimation": best_metrics["wi"],
        "initial_estimate_of_expected_n_ticks_in_bar": best_metrics["seed_ticks"],
    }

    # --------- decide whether to persist ----------
    chosen = _output(
        _old_param=_db_best_params,
        _new_param=new_params,
        param_db=params_resampling_db,
        sample_size=sample_size,
    )
    return chosen

## Optimized Sampling Schemes Parameters

### Optimized Imbalance Bars

In [19]:
max_batch_size = _batch_size(df_raw=df)

Bytes per row ≈ 165.0
Available RAM ≈ 16.2 GB
Safe batch_size ≈ 29,433,260 rows


In [20]:
n_trials = 100
n_pat = 60

##### Imbalance Tick Bars Parameters

In [ ]:
params_expected_tick_bars = sampling_best_params(sampling_method='expected_tick_imbalance_bars', 
                                                df_sample=df_riskAI, 
                                                params_resampling_db=db_resampling_params, 
                                                db_file_name=param_db_filename ,                                                
                                                chunk_size = max_batch_size , 
                                                trials=n_trials, 
                                                n_particles=n_pat,
                                                time_frame_label="1m"
)

params_expected_tick_bars

2025-09-15 20:10:33,792 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best:   0%|          |0/100

Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167


pyswarms.single.global_best:   1%|          |1/100, best_cost=8.42e+6

Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167


pyswarms.single.global_best:   2%|▏         |2/100, best_cost=8.4e+6 

...

Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 w

pyswarms.single.global_best:   9%|▉         |9/100, best_cost=8.4e+6

Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 w

pyswarms.single.global_best:  10%|█         |10/100, best_cost=8.4e+6

Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167
Processing batch 0 with size 39167


##### Imbalance Volume Bars Parameters

In [ ]:
params_expected_volume_bars = sampling_best_params(sampling_method='expected_volume_imbalance_bars', 
                                                df_sample=df_riskAI, 
                                                params_resampling_db=db_resampling_params, 
                                                db_file_name=param_db_filename,
                                                chunk_size = max_batch_size , 
                                                trials=n_trials, 
                                                n_particles=n_pat,
                                                time_frame_label="1m"
)

params_expected_volume_bars

##### Imbalance Dollar Bars Parameters

In [ ]:
params_expected_dollar_bars = sampling_best_params(sampling_method='expected_dollar_imbalance_bars', 
                                                df_sample=df_riskAI, 
                                                params_resampling_db=db_resampling_params, 
                                                db_file_name=param_db_filename ,
                                                chunk_size = max_batch_size , 
                                                trials=n_trials, 
                                                n_particles=n_pat,
                                                time_frame_label="1m"
)

params_expected_dollar_bars

### Optimized Run Bars

##### Run Bars - Tick Parameters

In [ ]:
params_run_tick_bars = sampling_best_params(sampling_method='expected_tick_run_bars', 
                                                df_sample=df_riskAI, 
                                                params_resampling_db=db_resampling_params, 
                                                db_file_name=param_db_filename,
                                                chunk_size = max_batch_size , 
                                                trials=n_trials, 
                                                n_particles=n_pat,
                                                time_frame_label="1m"
)

params_run_tick_bars

In [ ]:
params_run_volume_bars = sampling_best_params(sampling_method='expected_volume_run_bars', 
                                                df_sample=df_riskAI, 
                                                params_resampling_db=db_resampling_params, 
                                                db_file_name=param_db_filename,
                                                chunk_size = max_batch_size , 
                                                trials=n_trials, 
                                                n_particles=n_pat,
                                                time_frame_label="1m"
)

params_run_volume_bars

In [ ]:
params_run_dollar_bars = sampling_best_params(sampling_method='expected_dollar_run_bars', 
                                                df_sample=df_riskAI, 
                                                params_resampling_db=db_resampling_params, 
                                                db_file_name=param_db_filename,
                                                chunk_size = max_batch_size , 
                                                trials=n_trials, 
                                                n_particles=n_pat,
                                                time_frame_label="1m"
)

params_run_dollar_bars

### Reload Resampling Database

In [ ]:
reloaded_db_resampling_params = load_param_resampling_db(param_db_filename)
reloaded_db_resampling_params

##### Run Bars - Volume Parameters

##### Run Bars - Dollar Parameters

# Sampling schemes

In financial time series, sampling schemes determine how raw tick-level data (individual trades) are aggregated into bars (OHLC structures). Traditional time bars sample at fixed calendar intervals, but these often distort statistical properties by oversampling quiet periods and undersampling volatile ones.

To address this, López de Prado (2018) introduced alternative, `event-driven` bars that adapt to market activity. In this work, the focus is on:

- `Expected Imbalance Bars (EIBs)`
EIBs close a bar when the accumulated buy–sell volume imbalance exceeds an expected threshold, estimated dynamically from historical data. This produces bars of variable length that contain approximately equal amounts of information, improving stationarity and normality of returns. EIBs are particularly well suited for machine learning tasks that rely on balanced and stable input data.

- `Expected Run Bars (ERBs)`
ERBs close a bar when the number of consecutive buy or sell trades (a “run”) surpasses an expected run length, again estimated adaptively. This highlights periods of persistent order flow, often associated with informed trading or liquidity grabs. ERBs are especially valuable for detecting market microstructure patterns, such as those studied in Smart Money Concepts (SMC).

In [ ]:
def generate_info_driven_bars(sampling_method,df_sample,db_best_params):

    best_params = db_resampling_best_params(_sampling_method='expected_dollar_imbalance_bars',db_params=db_best_params)
    info_driven_bar = controller.handle_input_command(
    method_name=sampling_method,
    method_arguments={
        "window_size_for_expected_n_ticks_estimation": best_params['window_size_for_expected_n_ticks_estimation'],
        "window_size_for_expected_imbalance_estimation": best_params['window_size_for_expected_imbalance_estimation'],
        "initial_estimate_of_expected_n_ticks_in_bar": best_params['initial_estimate_of_expected_n_ticks_in_bar'],
    },
    input_data=df_sample,
    batch_size=best_params['batch_size'],
    )

    return info_driven_bar

In [ ]:
features = [
    'ohlc_trades','ohlc_taker_base','ohlc_taker_quote',
    'l1_bid','l1_ask','l1_mid','l1_spread',
    'l1_bid_qty','l1_ask_qty','l1_imbalance',
    'l2_bid_depth','l2_ask_depth','l2_depth_asymmetry',
    'l2_bid_vwap','l2_ask_vwap','l2_bid_slope','l2_ask_slope',
    'tr_volume_base','tr_volume_quote','tr_vwap','tr_buy_sell_imbalance',
    'spot_price','perp_mark_price','basis_abs','basis_pct',
    'funding_rate','next_funding_time_ms'
]


## Expected Imbalance Bars

### Imbalance Tick Bars

In [ ]:
EIB_ticks = generate_info_driven_bars(sampling_method='expected_tick_imbalance_bars',
                                      df_sample=df_riskAI,
                                      db_best_params=reloaded_db_resampling_params)

EIB_ticks

### Imbalance Volume Bars

In [ ]:
EIB_vol = generate_info_driven_bars(sampling_method='expected_volume_imbalance_bars',
                                      df_sample=df_riskAI,
                                      db_best_params=reloaded_db_resampling_params)
EIB_vol

### Imbalance Dollar Bars

In [ ]:
EIB_dollar = generate_info_driven_bars(sampling_method='expected_dollar_imbalance_bars',
                                      df_sample=df_riskAI,
                                      db_best_params=reloaded_db_resampling_params)
EIB_dollar



### Statistical Test

#### Log Return

In [ ]:
time_returns = log_return(df_riskAI['price'])
ticks_EIB_returns = log_return(EIB_ticks['Close'])
volume_EIB_returns = log_return(EIB_vol['Close'])
dollars_EIB_returns = log_return(EIB_dollar['Close'])

In [ ]:
def jb_report(name, s):
    # ensure 1-D array/Series
    s = pd.Series(s).astype(float)
    # drop non-finite
    s = s.replace([np.inf, -np.inf], np.nan).dropna()

    n = len(s)
    if n < 3:
        print(f"{name}: not enough data for Jarque–Bera (n={n})")
        return

    res = stats.jarque_bera(s)
    print(f"{name}: JB stat={res.statistic:.6g}, p={res.pvalue:.3g}, n={n}")


####  Jarque–Bera test statistic 

The `Jarque–Bera (JB) test` is used to check whether data follow a normal distribution by looking at skewness and kurtosis. In this test, smaller values are desirable because they indicate the data are closer to being normally distributed. For example, a statistic around 1 suggests the data are reasonably consistent with normality. A very large value, such as 6,633,374, strongly signals that the data deviate from normality, often due to heavy tails or asymmetry. In rare cases, the statistic can be 0, which occurs if the data have exactly zero skewness and a normal level of kurtosis, or if the dataset has no variation at all.

In [ ]:
# print("Jarque-Bera test statistic for time returns:", int(stats.jarque_bera(time_returns)[0]))
# print("Jarque-Bera test statistic for EIB tick returns:", int(stats.jarque_bera(ticks_EIB_returns)[0]))
# print("Jarque-Bera test statistic for EIB volume returns:", int(stats.jarque_bera(volume_EIB_returns)[0]))
# print("Jarque-Bera test statistic for EIB dollar returns:", int(stats.jarque_bera(dollars_EIB_returns)[0]))

jb_report("time returns", time_returns)
jb_report("EIB tick returns", ticks_EIB_returns)
jb_report("EIB volume returns", volume_EIB_returns)
jb_report("EIB dollar returns", dollars_EIB_returns)

#### Shapiro-Wilk Test

The `Shapiro–Wilk` test is a statistical method used to check whether a dataset follows a normal distribution. Unlike the Jarque–Bera test, which looks at skewness and kurtosis, the Shapiro–Wilk test directly compares the data to a perfectly normal shape. The test produces a statistic between 0 and 1, where values closer to 1 indicate the data are more consistent with normality. For example, a statistic of 0.98 would suggest the data are likely normal, while a much smaller value, such as 0.70, would indicate a strong departure from normality. The test also provides a p-value: if it is larger than 0.05, the data are considered roughly normal; if smaller, the data are unlikely to be normally distributed.

In [ ]:
def shapiro_report(name, s):
    s = pd.Series(s).astype(float)
    s = s.replace([np.inf, -np.inf], np.nan).dropna()
    n = len(s)

    if n < 3:
        print(f"{name}: not enough data for Shapiro–Wilk (n={n})")
        return

    # subsample if > 5000 (recommended by SciPy docs)
    if n > 5000:
        rng = np.random.default_rng(42)
        s = rng.choice(s, 5000, replace=False)

    stat, pval = stats.shapiro(s)
    print(f"{name}: W={stat:.6f}, p={pval:.3g}, n={n}")

# usage
shapiro_report("time returns", time_returns)
shapiro_report("EIB tick returns", ticks_EIB_returns)
shapiro_report("EIB volume returns", volume_EIB_returns)
shapiro_report("EIB dollar returns", dollars_EIB_returns)

# print("Shapiro-Wilk test statistic for time returns:", stats.shapiro(time_returns))
# print("Shapiro-Wilk test statistic for EIB tick returns:", stats.shapiro(ticks_EIB_returns))
# print("Shapiro-Wilk test statistic for EIB volume returns:", stats.shapiro(volume_EIB_returns))
# print("Shapiro-Wilk test statistic for EIB dollar returns:", stats.shapiro(dollars_EIB_returns))

#### Kernel Density Estimate (KDE) plot

A `Kernel Density Estimate (KDE) plot` is a smooth curve that shows the probability distribution of a dataset. It can be thought of as a smoothed version of a histogram, where the peaks indicate where the data are most concentrated and the shape of the curve shows how the values are distributed. KDE plots are often used to visually assess whether data resemble a normal distribution or display skewness, heavy tails, or multiple peaks.

In [ ]:
#Standardize Data 
time_standard = (time_returns - time_returns.mean()) / time_returns.std()
EIB_tick_standard = (ticks_EIB_returns - ticks_EIB_returns.mean()) / ticks_EIB_returns.std()
EIB_volume_standard = (volume_EIB_returns  - volume_EIB_returns.mean()) / volume_EIB_returns.std()
EIB_dollar_standard = (dollars_EIB_returns - dollars_EIB_returns.mean()) / dollars_EIB_returns.std()

In [ ]:
#Distribution Plot
plt.figure(figsize=(16, 12))
sns.kdeplot(time_standard, label="Time", color="red")
sns.kdeplot(EIB_tick_standard, label="Tick", color="blue")
sns.kdeplot(EIB_volume_standard, label="Volume", color="green")
sns.kdeplot(EIB_dollar_standard, label="Dollar", color="purple", linestyle="-.")
sns.kdeplot(np.random.normal(size=1000000), label="Normal", linestyle="dotted")
plt.xticks(range(-4, +4))

#labels
plt.xlabel("Standardized Log Returns")
plt.ylabel("Density")
plt.title(
    'Partial Recovery of Normality for Expected Imbalance Bars',
    loc='center', 
)
plt.xlim(-5, 5)
plt.legend()
plt.show()

## Run Bars

### Tick Run Bars

In [ ]:
tick_run_bars = generate_info_driven_bars(sampling_method='expected_tick_run_bars',
                                      df_sample=df_riskAI,
                                      db_best_params=reloaded_db_resampling_params)
tick_run_bars

### Volume Run Bars

In [ ]:


volume_run_bars = generate_info_driven_bars(sampling_method='expected_volume_run_bars',
                                      df_sample=df_riskAI,
                                      db_best_params=reloaded_db_resampling_params)
volume_run_bars

### Dollar Run Bars

In [ ]:
dollar_run_bars = generate_info_driven_bars(sampling_method='expected_dollar_run_bars',
                                      df_sample=df_riskAI,
                                      db_best_params=reloaded_db_resampling_params)
dollar_run_bars

### Statistical Test

#### Log Return

In [ ]:
tick_run_bars_returns = log_return(tick_run_bars['Close'])
volume_run_bars_returns = log_return(volume_run_bars['Close'])
dollar_run_bars_returns = log_return(dollar_run_bars['Close'])

####  Jarque–Bera test statistic 

In [ ]:
print("Jarque-Bera test statistic for time returns:", int(stats.jarque_bera(time_returns)[0]))
print("Jarque-Bera test statistic for tick run bars returns:", int(stats.jarque_bera(tick_run_bars_returns)[0]))
print("Jarque-Bera test statistic for volume run bars returns:", int(stats.jarque_bera(volume_run_bars_returns)[0]))
print("Jarque-Bera test statistic for dollar run bars returns:", int(stats.jarque_bera(dollar_run_bars_returns)[0]))

#### Shapiro-Wilk Test

In [ ]:
print("Shapiro-Wilk test statistic for time returns:", stats.shapiro(time_returns))
print("Shapiro-Wilk test statistic for tick run bars returns:", stats.shapiro(tick_run_bars_returns))
print("Shapiro-Wilk test statistic for volume run bars returns:", stats.shapiro(volume_run_bars_returns))
print("Shapiro-Wilk test statistic for dollar run bars returns:", stats.shapiro(dollar_run_bars_returns))

#### Kernel Density Estimate (KDE) plot

In [ ]:
#Standardize Data 
tick_run_bars_standard = (tick_run_bars_returns - tick_run_bars_returns.mean()) / tick_run_bars_returns.std()
volume_run_bars_standard = (volume_run_bars_returns  - volume_run_bars_returns.mean()) / volume_run_bars_returns.std()
dollar_run_bars_standard = (dollar_run_bars_returns - dollar_run_bars_returns.mean()) / dollar_run_bars_returns.std()

In [ ]:
#Distribution Plot
plt.figure(figsize=(16, 12))
sns.kdeplot(time_standard, label="Time", color="red")
sns.kdeplot(tick_run_bars_standard, label="Tick", color="blue")
sns.kdeplot(volume_run_bars_standard, label="Volume", color="green")
sns.kdeplot(dollar_run_bars_standard , label="Dollar", color="purple", linestyle="-.")
sns.kdeplot(np.random.normal(size=1000000), label="Normal", linestyle="dotted")
plt.xticks(range(-4, +4))

#labels
plt.xlabel("Standardized Log Returns")
plt.ylabel("Density")
plt.title(
    'Partial Recovery of Normality for Run Bars',
    loc='center', 
)
plt.xlim(-5, 5)
plt.legend()
plt.show()

# Additional Features

In [ ]:
def grouped_features(raw_df, resumpling_df, feature_name, agg_func=np.sum):
    """
    Aggregate a feature between Tick Number ranges using a specified aggregation function.

    Parameters
    ----------
    raw_df : pd.DataFrame
        Full DataFrame with tick-by-tick data.
    resumpling_df : pd.DataFrame
        DataFrame with "Tick Number" boundaries (breakpoints).
    feature_name : str
        Column name in raw_df to aggregate (e.g., "ohlc_trades").
    agg_func : function, default=np.sum
        Aggregation function (e.g., np.sum, np.mean, np.max).

    Returns
    -------
    pd.DataFrame
        resumpling_df with the aggregated feature filled.
    """

    resumpling_df[feature_name] = np.nan  

    m = raw_df[feature_name]
    idx_list = resumpling_df.index.astype(int).to_list()

    if idx_list:
        idx_start = idx_list[0]
        idx_end = idx_list[0] + 1
        resumpling_df.loc[idx_start, feature_name] = agg_func(m.iloc[0:idx_end])

    for start, end in zip(idx_list, idx_list[1:]):
        resumpling_df.loc[end, feature_name] = agg_func(m.iloc[start:(end+1)])

    return resumpling_df

def last_state(raw_df,resumpling_df, feature_name):
    """
    Align the latest state of a feature from the raw dataframe
    onto the resampled dataframe at matching indices.

    Parameters
    ----------
    raw_df : pandas.DataFrame
        Original/raw dataset containing the feature of interest.
    resampling_df : pandas.DataFrame
        Resampled dataset whose index is aligned to raw_df.
    feature_name : str
        Column name (feature) to propagate from raw_df to resampling_df.

    Returns
    -------
    pandas.DataFrame
        Updated resampling_df with a new column `feature_name`
        containing the values from raw_df at matching indices. 

    """

    resumpling_df[feature_name] = np.nan  
    idx_list = resumpling_df.index.to_list()
    for i in idx_list:
        resumpling_df.loc[i,feature_name] = raw_df[feature_name][i]
    return resumpling_df

def price_vwap(raw_df, resumpling_df, feature_name, vol_colum_name ='ohlc_volume'):
    """
    Compute VWAP (Volume-Weighted Average Price) for each resampled bar.

    Parameters
    ----------
    raw_df : pandas.DataFrame
        Original dataframe with price and volume data.
    resampling_df : pandas.DataFrame
        Resampled dataframe that defines the bar boundaries (by index).
    feature_name : str
        Column name in raw_df containing prices to weight.
    vol_column_name : str, default="ohlc_volume"
        Column name in raw_df containing volume weights.

    Returns
    -------
    pandas.DataFrame
        Updated resampling_df with an extra column containing VWAP values
        for each resampled interval.
    """
    
    resumpling_df[feature_name] = np.nan  

    m = raw_df[feature_name]
    vol = raw_df[vol_colum_name]

    idx_list = resumpling_df.index.astype(int).to_list()

    if idx_list:
        idx_start = idx_list[0]
        idx_end = idx_list[0] + 1

        resumpling_df.loc[idx_start, feature_name] = np.average(m[0:idx_end], weights=vol[0:idx_end])

    for start, end in zip(idx_list, idx_list[1:]):
        resumpling_df.loc[end, feature_name] = np.average(m[start:(end+1)], weights=vol[start:(end+1)])

    return resumpling_df

In [ ]:
def aggregate_features(input_df, sampling_df):

    """
    Aggregate raw features into resampled bars using different strategies.

    Parameters
    ----------
    input_df : pandas.DataFrame
        Raw tick-level dataframe containing all features.
    sampling_df : pandas.DataFrame
        DataFrame with resampled bar boundaries.
        Must contain a 'Tick Number' column to align with input_df.

    Returns
    -------
    pandas.DataFrame
        Updated sampling_df containing aggregated features.
    """

    sampling_df =sampling_df.set_index("Tick Number")

    vol_sum = ['ohlc_trades','ohlc_taker_base','ohlc_taker_quote',
            'tr_volume_base','tr_volume_quote']

    last_states = ['l1_bid','l1_ask','l1_mid','l1_spread',
                'l1_bid_qty','l1_ask_qty','l1_imbalance',
                'l2_bid_depth','l2_ask_depth','l2_depth_asymmetry',
                'l2_bid_vwap','l2_ask_vwap','l2_bid_slope','l2_ask_slope',
                'spot_price','perp_mark_price','basis_abs','basis_pct',
                'funding_rate','next_funding_time_ms']

    mean_bar= ['tr_buy_sell_imbalance']

    weighted_mean = ['tr_vwap']

    #total
    for i in vol_sum:
        sampling_df  = grouped_features(raw_df=input_df, 
                                        resumpling_df=sampling_df , 
                                        feature_name=i, 
                                        agg_func=np.sum)
    #last value
    for j in last_states:
        sampling_df  = last_state(raw_df=input_df,
                                    resumpling_df=sampling_df ,
                                    feature_name=j)
    #average value   
    for k in mean_bar:
        sampling_df  = grouped_features(raw_df=input_df, 
                                        resumpling_df=sampling_df , 
                                        feature_name=k, 
                                        agg_func=np.mean)
    #weighted average
    for _ in weighted_mean:
        sampling_df  = price_vwap(raw_df=input_df, 
                                    resumpling_df=sampling_df , 
                                    feature_name= _,
                                    vol_colum_name ='ohlc_volume')
        
    return sampling_df

In [ ]:
df_features = aggregate_features(input_df=df, sampling_df=dollar_run_bars)
df_features